In [4]:
# Application
import cv2
from tensorflow import keras
from keras.models import load_model
import numpy as np
from collections import deque # queue

d:\anaconda3\envs\tensorenviron\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\anaconda3\envs\tensorenviron\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\anaconda3\envs\tensorenviron\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\anaconda3\envs\tensorenviron\lib\site-packages\tensorflow\python\framework\dtypes.

In [5]:
model1 = load_model('devnagari.h5')
print(model1)

In [6]:
letter_count={0:'CHECK',1:'character_01_ka',2:'character_02_kha', 3:'character_03_ga', 4:'character_04_gha',
              5:'character_05_kna', 6:'character_06_cha', 7:'character_07_chha',
             8:'character_08_ja', 9:'character_09_jha', 10:'character_10_yna', 11:'character_11_taamatar',
             12:'character_12_thaa', 13:'character_13_daa', 14:'character_14_dhaa', 15:'character_15_adna',
              16:'character_16_tabala', 17:'character_17_tha', 18:'character_18_da',
             19:'character_19_dha',
             20:'character_20_na', 21:'character_21_pa', 22:'character_22_pha', 23:'character_23_ba',
             24:'character_24_bha',
             25:'character_25_ma', 26:'character_26_yaw', 27:'character_27_ra', 28:'character_28_la',
              29:'character_29_waw', 30:'character_30_motosaw', 31:'character_31_petchiryakha',
             32:'character_32_patalosaw', 33:'character_33_ha', 34:'character_34_chhya',
             35:'character_35_tra', 36:'character_36_gya', 37:'digit_0',
             38:'digit_1', 39:'digit_2', 40:'digit_3', 41:'digit_4', 42:'digit_5',
             43:'digit_6', 44:'digit_7', 45:'digit_8', 46:'digit_9', 47:'CHECK'}

In [7]:
def keras_predict(model,image):
    processed = keras_process_image(image)
    print("Processed:",str(processed.shape))
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab),pred_class

def keras_process_image(img):
    img_x = 32
    img_y = 32
    img = cv2.resize(img, (img_x, img_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img,(-1, img_x, img_y, 1))
    return img

In [20]:
cap = cv2.VideoCapture(0)
Lower_blue = np.array([110, 50, 50])
Upper_blue = np.array([130, 255, 255])
pred_class = 0
pts = deque(maxlen=512)
blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
digit = np.zeros((200, 200, 3), dtype=np.uint8)
while(cap.isOpened()):
    ret, img = cap.read()
    img = cv2.flip(img, 1)
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(imgHSV, Lower_blue, Upper_blue)
    blur = cv2.medianBlur(mask, 15)
    blur = cv2.GaussianBlur(blur, (5, 5), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
    center = None
    if(len(cnts)>=1):
        contour = max(cnts, key=cv2.contourArea)
        if cv2.contourArea(contour)>250:
            ((x,y), radius) = cv2.minEnclosingCircle(contour)
            cv2.circle(img, (int(x), int(y)), int(radius), (0, 255, 255), 2)
            cv2.circle(img, center, 5, (0, 0, 255), -1)
            M = cv2.moments(contour)
            center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
            pts.appendleft(center)
            for i in range(1, len(pts)):
                if pts[i-1] is None or pts[i] is None:
                    continue
                cv2.line(blackboard, pts[i-1], pts[i], (255, 255, 255), 10)
                cv2.line(img, pts[i-1], pts[i], (0, 0, 255), 5)
    elif len(cnts) == 0:
        if len(pts) !=[]:
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur1 = cv2.medianBlur(blackboard_gray, 15)
            blur1 = cv2.GaussianBlur(blur1, (5,5), 0)
            thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            blackboard_cnts = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
            if len(blackboard_cnts) >=1:
                cnt = max(blackboard_cnts, key = cv2.contourArea)
                print(cv2.contourArea(cnt))
                if cv2.contourArea(cnt) > 2000:
                    x, y, w, h = cv2.boundingRect(cnt)
                    digit = blackboard_gray[y:y + h, x:x + w]
                    # newImage = process_letter(digit)
                    pred_probab, pred_class = keras_predict(model1, digit)
                    print(pred_class, pred_probab)
                        
        pts = deque(maxlen=512)
        blackboard = np.zeros((480, 640, 3), dtype = np.uint8)
    cv2.putText(img, "Conv Network: "+str(letter_count[pred_class]), (10, 470),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow("Frame", img)
    cv2.imshow("Contours", thresh)
    k = cv2.waitKey(10)
    if k==27:
        break
        
cv2.destroyAllWindows()
cap.release()

682.0
3422.5
Processed: (1, 32, 32, 1)
43 1.0
342.0
16271.5
Processed: (1, 32, 32, 1)
39 1.0
6921.0
Processed: (1, 32, 32, 1)
33 1.0
250.5
4819.0
Processed: (1, 32, 32, 1)
27 1.0
7957.0
Processed: (1, 32, 32, 1)
33 1.0
17637.0
Processed: (1, 32, 32, 1)
42 1.0
28206.0
Processed: (1, 32, 32, 1)
39 1.0
13716.5
Processed: (1, 32, 32, 1)
23 1.0
783.0
315.0
3429.0
Processed: (1, 32, 32, 1)
8 1.0
9334.5
Processed: (1, 32, 32, 1)
4 1.0
6831.0
Processed: (1, 32, 32, 1)
8 1.0
6909.5
Processed: (1, 32, 32, 1)
6 1.0
48.0
27334.5
Processed: (1, 32, 32, 1)
8 1.0
26193.0
Processed: (1, 32, 32, 1)
0 1.0
26282.5
Processed: (1, 32, 32, 1)
33 1.0
25215.0
Processed: (1, 32, 32, 1)
23 1.0
11197.0
Processed: (1, 32, 32, 1)
8 1.0
12054.5
Processed: (1, 32, 32, 1)
38 1.0
15047.0
Processed: (1, 32, 32, 1)
22 1.0
8005.0
Processed: (1, 32, 32, 1)
38 1.0
9837.5
Processed: (1, 32, 32, 1)
1 1.0
1092.5
2240.0
Processed: (1, 32, 32, 1)
43 1.0
10820.5
Processed: (1, 32, 32, 1)
39 1.0
674.5
3883.0
Processed: (1, 32, 32